This code was written to analyse the E. coli model with and without constriants.
We started by loading the model and optimizing to get the fluxes of the model
We then applied the constraint in different conditions:

Ec = WT + Ecoli (the both of them)

E = Ecoli alone

W = WT + Ecoli (LATE)

K = dKil + Ecoli

In [90]:
#importing the needed packages to study the model
import cobra
import pandas as pd

In [91]:
#loading mdoel
model = cobra.io.read_sbml_model("iML1515.xml")

After having loaded the model, we want to optimize it with no constraints to see how the model works and the fluxes of the model 

In [92]:
#Getting the optimized model in variable 
model_optimal = model.optimize()
model_optimal

,fluxes,reduced_costs
CYTDK2,0.000000e+00,9.521763e-17
XPPT,-8.698806e-17,1.387779e-17
HXPRT,0.000000e+00,-1.504671e-02
NDPK5,0.000000e+00,3.855759e-16
SHK3Dr,3.342403e-01,-2.639676e-18
...,...,...
MPTS,0.000000e+00,-2.602085e-18
MOCOS,0.000000e+00,-5.642514e-03
BMOGDS2,0.000000e+00,-3.338488e-01
FESD2s,0.000000e+00,-3.447763e-17


In [93]:
#Storing the fluxes from the table in a new variable
model_optimal_fluxes= model_optimal.fluxes
model_optimal_fluxes

CYTDK2     0.000000e+00
XPPT      -8.698806e-17
HXPRT      0.000000e+00
NDPK5      0.000000e+00
SHK3Dr     3.342403e-01
               ...     
MPTS       0.000000e+00
MOCOS      0.000000e+00
BMOGDS2    0.000000e+00
FESD2s     0.000000e+00
OCTNLL     0.000000e+00
Name: fluxes, Length: 2712, dtype: float64

This function was made to set constrains on the model by taking the dictionary of normalized gene expressions

In [94]:
model1 = cobra.io.read_sbml_model("iML1515.xml")
def Eflux(model,dict_expression,default_value):

    """Function used to do the Eflux

    

    Args:

        model (cobra.core.model.Model): A cobrapy model

        dict_expression (dict): Dict of normalized RNAseq result, the key is the gene ID and the value is the expression level

        default_value (int): Default value for the upper bound, it will be scaled with the expression level.

    

    """

    rlist = [] # list used to work with reaction with two genes
    lp = 0.9 # not useful anymore (At first I was also using the lower bound)
    up = 1.1 # add 10% to the upper bound
    for gene in model.genes: 
        if gene.id in dict_expression.keys(): # If the gene is in the RNA-seq data
            #print(f"gene : {gene}")
            #print(f"LB = dv({dv}) * lp({lp}) * de({dict_exp[gene.id]})")
            #lb = dv * lp * dict_exp[gene.id]
            #print(f"UB = dv({dv}) * lp({up}) * de({dict_exp[gene.id]})\n")
            ub = default_value * up * dict_expression[gene.id] # The new upper bound is calculated for this gene
            for react in gene.reactions: # For each reaction linked to this gene
                if react in model.exchanges:
                    continue
                if react.id in rlist: # If another gene is linked to it:
                    if react.upper_bound < ub: # use the maximum upper bound
                        react.upper_bound = ub
                        if react.lower_bound < 0: # and if it's reversible set the lower bound
                            react.lower_bound = -ub
                else: # If there is no other gene (or if it's the first time)
                    if react.lower_bound < 0: # if the reaction is reversible :
                        react.lower_bound = -ub # set the lower bound
                    react.upper_bound = ub # set the upper bound
                    rlist.append(react.id) #add the react in the list to see if there is another gene


This functuon was made to turn the normalized count table into a dictionary

In [95]:
def dict_maker(normalized_counts):
    """
    Takes as args a table to turn the table into a normalized dict
    
    Args:
        normalized_counts : table of normalized counts to set as constraints
    """
    
    #puts table in dataframe
    normalized_counts.iloc[:,1] = normalized_counts.iloc[:,1]/max(normalized_counts.iloc[:,1])
    df = pd.DataFrame(normalized_counts)
    #transpose the data frame to be able to concat later
    df1 = df.T
    df1.columns = df1.iloc[0]
    df1 = df1.tail(-1)
#     df1[2]= df1[2]/max(df1[2])
    #changing the data frame to dictionary 
    normalized_dict= df1.to_dict("list")
    return normalized_dict

Applying the function to all the tables we have 

In [96]:
normalized_table_W = pd.read_csv("/home/lynn/Desktop/internship/modelling/test/Internship_code/normalized_counts_W.csv")
normalized_dict_W = dict_maker(normalized_table_W)
normalized_table_Ec = pd.read_csv("/home/lynn/Desktop/internship/modelling/test/Internship_code/normalized_counts_Ec.csv")
normalized_dict_Ec = dict_maker(normalized_table_Ec)
normalized_table_E = pd.read_csv("/home/lynn/Desktop/internship/modelling/test/Internship_code/normalized_counts_E.csv")
normalized_dict_E = dict_maker(normalized_table_E)
normalized_table_E_C1 = pd.read_csv("/home/lynn/Desktop/internship/modelling/test/Internship_code/normalized_counts_E_C1.csv")
normalized_dict_E_C1 = dict_maker(normalized_table_E_C1)
normalized_table_E_C2 = pd.read_csv("/home/lynn/Desktop/internship/modelling/test/Internship_code/normalized_counts_E_C2.csv")
normalized_dict_E_C2 = dict_maker(normalized_table_E_C2)

In [97]:
normalized_table_Ec

,Unnamed: 0,Mean_Ec
0,b0001,0.169450
1,b0002,0.601687
2,b0003,0.523560
3,b0004,0.527413
4,b0005,0.286032
...,...,...
4300,b4400,0.254826
4301,b4401,0.491572
4302,b4402,0.436532
4303,b4824,0.290760


Chaging the dictionary to set the gene as key and expression as value
Storing variable of optimized model for each constrain type

In [98]:
for i in normalized_dict_W:
    normalized_dict_W[i]=normalized_dict_W[i][0]

In [99]:
normalized_dict_W

{'b0001': 0.1864794460375254,
 'b0002': 0.4483874295719801,
 'b0003': 0.2538950996321129,
 'b0004': 0.2956325410454372,
 'b0005': 0.23384642460222071,
 'b0006': 0.2989282229006354,
 'b0007': 0.31750043748221674,
 'b0008': 0.5539683177582088,
 'b0009': 0.33504232496987213,
 'b0010': 0.355644956100812,
 'b0011': 0.35800492808120193,
 'b0012': 0.16608416739424858,
 'b0013': 0.1864794460375254,
 'b0014': 0.5272771065753171,
 'b0015': 0.28787763396495675,
 'b0016': 0.3130083352972385,
 'b0018': 0.16608416739424858,
 'b4412': 0.16608416739424858,
 'b0019': 0.45536779354994117,
 'b0020': 0.23384642460222071,
 'b0021': 0.16608416739424858,
 'b0022': 0.16608416739424858,
 'b0023': 0.4658633700538854,
 'b0024': 0.42355896035598145,
 'b0025': 0.3795854215529216,
 'b0026': 0.5859053965674429,
 'b0027': 0.422321289177267,
 'b0028': 0.4130471377230452,
 'b0029': 0.4359506510578906,
 'b0030': 0.16608416739424858,
 'b0031': 0.3364556697801836,
 'b0032': 0.5806294876531114,
 'b0033': 0.6778958793131213

In [100]:
W = Eflux(model1, normalized_dict_W, 100)
model.optimal_W = model1.optimize()
model.optimal_W

,fluxes,reduced_costs
CYTDK2,0.000000,-7.273028e-17
XPPT,0.000000,-1.387779e-17
HXPRT,0.000000,-1.661520e-02
NDPK5,0.020273,-4.819699e-17
SHK3Dr,0.285991,-4.199484e-18
...,...,...
MPTS,0.000000,4.076600e-17
MOCOS,0.000000,-8.307601e-03
BMOGDS2,0.000000,-4.301780e-01
FESD2s,0.000000,-4.361490e-02


In [101]:
for i in normalized_dict_E:
    normalized_dict_E[i]=normalized_dict_E[i][0]
normalized_dict_E

{'b0001': 0.26329489416138796,
 'b0002': 0.6128051508001856,
 'b0003': 0.5224520848857208,
 'b0004': 0.5313258566784537,
 'b0005': 0.32803026529376234,
 'b0006': 0.4369902664353462,
 'b0007': 0.41529173722083373,
 'b0008': 0.7085529003914873,
 'b0009': 0.47088151890304936,
 'b0010': 0.31897587280139705,
 'b0011': 0.10432043608750785,
 'b0012': 0.0004051091987846744,
 'b0013': 0.25730936883657785,
 'b0014': 0.5913736048777284,
 'b0015': 0.5179942858693735,
 'b0016': 0.4709784344294382,
 'b0018': 0.19049873867972378,
 'b4412': 0.0004051091987846744,
 'b0019': 0.6899628272921474,
 'b0020': 0.5396326045096325,
 'b0021': 0.36860224545508813,
 'b0022': 0.3363977873003928,
 'b0023': 0.5106261235890882,
 'b0024': 0.4143166305286684,
 'b0025': 0.6266703686969249,
 'b0026': 0.7449622981148217,
 'b0027': 0.4150449939405752,
 'b0028': 0.512900718181331,
 'b0029': 0.5598175460395577,
 'b0030': 0.25386506047417984,
 'b0031': 0.5481473093690702,
 'b0032': 0.6982356628489337,
 'b0033': 0.7254792480479

In [102]:
E = Eflux(model1, normalized_dict_E, 100)
model.optimal_E = model1.optimize()
model.optimal_E

,fluxes,reduced_costs
CYTDK2,0.000000e+00,4.497471e-17
XPPT,0.000000e+00,0.000000e+00
HXPRT,0.000000e+00,-1.504672e-02
NDPK5,-9.608357e-16,-0.000000e+00
SHK3Dr,3.342407e-01,1.943761e-17
...,...,...
MPTS,0.000000e+00,-1.127570e-17
MOCOS,0.000000e+00,-3.413725e-01
BMOGDS2,0.000000e+00,-3.885781e-16
FESD2s,0.000000e+00,-1.178745e-15


In [ ]:
for i in normalized_dict_Ec:
    normalized_dict_Ec[i]=normalized_dict_Ec[i][0]
normalized_dict_Ec

In [84]:
Ec = Eflux(model1, normalized_dict_Ec, 100)
model.optimal_Ec = model1.optimize()
model.optimal_Ec

,fluxes,reduced_costs
CYTDK2,0.000000e+00,-1.834476e-02
XPPT,0.000000e+00,-3.668953e-02
HXPRT,0.000000e+00,-3.668953e-02
NDPK5,4.719783e-19,-0.000000e+00
SHK3Dr,5.632812e-02,1.161657e-17
...,...,...
MPTS,0.000000e+00,4.484525e-17
MOCOS,0.000000e+00,-3.791251e-01
BMOGDS2,0.000000e+00,2.775558e-17
FESD2s,0.000000e+00,-4.280445e-02


In [15]:
for i in normalized_dict_E_C1:
    normalized_dict_E_C1[i]=normalized_dict_E_C1[i][0]
normalized_dict_E_C1

{'b0001': 3.31469652565629,
 'b0002': 8.21795769786411,
 'b0003': 7.03913839390696,
 'b0004': 7.22583403553784,
 'b0005': 4.30669970727279,
 'b0006': 5.84549005094437,
 'b0007': 3.63575439127776,
 'b0008': 9.62007316544424,
 'b0009': 7.15167619581708,
 'b0010': 4.99412711400313,
 'b0011': 0.787819021371666,
 'b0012': 0.0404636846081765,
 'b0013': 1.89141918684608,
 'b0014': 7.85337194086739,
 'b0015': 6.75595576955096,
 'b0016': 4.51096191927738,
 'b0018': 1.62714584316111,
 'b4412': 0.0404636846081765,
 'b0019': 7.68243284122705,
 'b0020': 5.16631374788993,
 'b0021': 3.47767732756531,
 'b0022': 3.38266725274504,
 'b0023': 9.17819175867861,
 'b0024': 6.76128527336162,
 'b0025': 7.31070365722511,
 'b0026': 11.4925992976279,
 'b0027': 6.42105494692424,
 'b0028': 7.16912328147676,
 'b0029': 8.0092127868052,
 'b0030': 3.38956681176273,
 'b0031': 6.54303182025524,
 'b0032': 12.4861964751488,
 'b0033': 13.6578509871448,
 'b0034': 7.21674585819531,
 'b0035': 1.52105073690096,
 'b0036': 0.7311

In [16]:
E_C1 = Eflux(model1, normalized_dict_E_C1, 100)
model.optimal_E_C1 = model1.optimize()
model.optimal_E_C1

,fluxes,reduced_costs
CYTDK2,0.000000,-4.497471e-17
XPPT,0.000000,0.000000e+00
HXPRT,0.000000,-1.665139e-02
NDPK5,0.000000,-0.000000e+00
SHK3Dr,0.315389,-3.683547e-17
...,...,...
MPTS,0.000000,9.801188e-17
MOCOS,0.000000,-8.325693e-03
BMOGDS2,0.000000,-4.131625e-01
FESD2s,0.000000,-2.289566e-02


In [17]:
for i in normalized_dict_E_C2:
    normalized_dict_E_C2[i]=normalized_dict_E_C2[i][0]
normalized_dict_E_C2

{'b0001': 4.27202318906105,
 'b0002': 8.71932036403724,
 'b0003': 10.6913073301911,
 'b0004': 7.35499863931907,
 'b0005': 4.3743439890385,
 'b0006': 9.21977355089287,
 'b0007': 4.15588959451476,
 'b0008': 9.40830850889606,
 'b0009': 10.2744713400658,
 'b0010': 5.52042224852645,
 'b0011': 1.48334442967338,
 'b0012': 0.0375459484678048,
 'b0013': 2.10433665981474,
 'b0014': 8.01100356443267,
 'b0015': 10.3102719472978,
 'b0016': 5.84649331596456,
 'b0018': 1.52606881166759,
 'b4412': 0.0375459484678048,
 'b0019': 7.77623561482759,
 'b0020': 5.38370429247405,
 'b0021': 7.44418666269936,
 'b0022': 5.01346225980656,
 'b0023': 9.66204523878578,
 'b0024': 11.2438225742894,
 'b0025': 8.1811025507538,
 'b0026': 10.4050137853593,
 'b0027': 9.73940934222919,
 'b0028': 7.45343526884372,
 'b0029': 7.98384867402245,
 'b0030': 7.19199684666977,
 'b0031': 6.81890167588564,
 'b0032': 12.1375163317101,
 'b0033': 16.8189011767246,
 'b0034': 7.36045207176887,
 'b0035': 2.02856915219677,
 'b0036': 4.940678

In [18]:
E_C2 = Eflux(model1, normalized_dict_E_C2, 100)
model.optimal_E_C2 = model1.optimize()
model.optimal_E_C2

,fluxes,reduced_costs
CYTDK2,0.000000e+00,-2.775558e-17
XPPT,0.000000e+00,-1.387779e-17
HXPRT,0.000000e+00,-1.504672e-02
NDPK5,2.161838e-18,-0.000000e+00
SHK3Dr,3.342407e-01,-5.243356e-17
...,...,...
MPTS,0.000000e+00,-5.030698e-17
MOCOS,0.000000e+00,-3.413725e-01
BMOGDS2,0.000000e+00,2.775558e-17
FESD2s,0.000000e+00,-5.811324e-16


Making a data frame of all the fluxes by putting them in a list first

In [104]:
fluxes_no_constraints = model_optimal.fluxes
fluxes_W = model.optimal_W.fluxes
# fluxes_Ec = model.optimal_Ec.fluxes
fluxes_E = model.optimal_E.fluxes
# fluxes_E_C1 = model.optimal_E_C1.fluxes
# fluxes_E_C2 = model.optimal_E_C2.fluxes
all_fluxes = [fluxes_no_constraints, fluxes_W, fluxes_E]

In [105]:
df = pd.DataFrame(all_fluxes)
df

,CYTDK2,XPPT,HXPRT,NDPK5,SHK3Dr,NDPK6,NDPK8,DHORTS,OMPDC,PYNP2r,...,COLIPAabctex,ENLIPAabctex,DM_mththf_c,PYROX,BMOGDS1,MPTS,MOCOS,BMOGDS2,FESD2s,OCTNLL
fluxes,0.0,-8.698806e-17,0.0,0.000000e+00,0.334240,0.000000,5.929692e-16,-0.290106,0.290106,2.320660e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fluxes,0.0,0.000000e+00,0.0,2.027348e-02,0.285991,0.019635,1.963489e-02,-0.248228,0.248228,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fluxes,0.0,0.000000e+00,0.0,-9.608357e-16,0.334241,0.000000,0.000000e+00,-0.290107,0.290107,9.909925e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
df = df.set_axis(['Fluxes no constraints', 'Fluxes W', 'Fluxes E'], axis='index')
df

,CYTDK2,XPPT,HXPRT,NDPK5,SHK3Dr,NDPK6,NDPK8,DHORTS,OMPDC,PYNP2r,...,COLIPAabctex,ENLIPAabctex,DM_mththf_c,PYROX,BMOGDS1,MPTS,MOCOS,BMOGDS2,FESD2s,OCTNLL
Fluxes no constraints,0.0,-8.698806e-17,0.0,0.000000e+00,0.334240,0.000000,5.929692e-16,-0.290106,0.290106,2.320660e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fluxes W,0.0,0.000000e+00,0.0,2.027348e-02,0.285991,0.019635,1.963489e-02,-0.248228,0.248228,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fluxes E,0.0,0.000000e+00,0.0,-9.608357e-16,0.334241,0.000000,0.000000e+00,-0.290107,0.290107,9.909925e-16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
df1 = df[df.columns[df.sum()>0.0]]
df1

,NDPK5,SHK3Dr,NDPK6,NDPK8,OMPDC,PYNP2r,G5SD,CS,ICDHyr,PPA,...,PAI2I,FESD1s,I2FE2SR,TYRL,THZPSN3,PAI2T,APG3PAT120,4HTHRK,APH120,BIOMASS_Ec_iML1515_core_75p37M
Fluxes no constraints,0.000000e+00,0.334240,0.000000,5.929692e-16,0.290106,2.320660e-16,0.193865,6.913003,6.913003,3.070861,...,0.000393,0.003424,0.000002,0.000196,0.000196,0.000393,1.731617e-17,0.000587,3.671980e-17,0.876997
Fluxes W,2.027348e-02,0.285991,0.019635,1.963489e-02,0.248228,0.000000e+00,0.165879,7.793504,5.572162,2.627562,...,0.000336,0.000000,0.000002,0.000167,0.000167,0.000336,0.000000e+00,0.000502,0.000000e+00,0.750397
Fluxes E,-9.608357e-16,0.334241,0.000000,0.000000e+00,0.290107,9.909925e-16,0.193865,6.911908,6.911908,3.070865,...,0.000393,0.003424,0.000002,0.000196,0.000196,0.000393,0.000000e+00,0.000587,0.000000e+00,0.876998


In [108]:
df1.to_csv("fluxes_results.csv")

In [109]:
df1_diff = pd.DataFrame(df1.iloc[1] - df1.iloc[2]).T
df1_diff

,NDPK5,SHK3Dr,NDPK6,NDPK8,OMPDC,PYNP2r,G5SD,CS,ICDHyr,PPA,...,PAI2I,FESD1s,I2FE2SR,TYRL,THZPSN3,PAI2T,APG3PAT120,4HTHRK,APH120,BIOMASS_Ec_iML1515_core_75p37M
0,0.020273,-0.04825,0.019635,0.019635,-0.041879,-9.909925e-16,-0.027986,0.881597,-1.339746,-0.443302,...,-0.000057,-0.003424,-2.532024e-07,-0.000028,-0.000028,-0.000057,0.0,-0.000085,0.0,-0.126601


In [110]:
df2 = pd.concat([df1, df1_diff], ignore_index=False)
df2

,NDPK5,SHK3Dr,NDPK6,NDPK8,OMPDC,PYNP2r,G5SD,CS,ICDHyr,PPA,...,PAI2I,FESD1s,I2FE2SR,TYRL,THZPSN3,PAI2T,APG3PAT120,4HTHRK,APH120,BIOMASS_Ec_iML1515_core_75p37M
Fluxes no constraints,0.000000e+00,0.334240,0.000000,5.929692e-16,0.290106,2.320660e-16,0.193865,6.913003,6.913003,3.070861,...,0.000393,0.003424,1.753994e-06,0.000196,0.000196,0.000393,1.731617e-17,0.000587,3.671980e-17,0.876997
Fluxes W,2.027348e-02,0.285991,0.019635,1.963489e-02,0.248228,0.000000e+00,0.165879,7.793504,5.572162,2.627562,...,0.000336,0.000000,1.500794e-06,0.000167,0.000167,0.000336,0.000000e+00,0.000502,0.000000e+00,0.750397
Fluxes E,-9.608357e-16,0.334241,0.000000,0.000000e+00,0.290107,9.909925e-16,0.193865,6.911908,6.911908,3.070865,...,0.000393,0.003424,1.753996e-06,0.000196,0.000196,0.000393,0.000000e+00,0.000587,0.000000e+00,0.876998
0,2.027348e-02,-0.048250,0.019635,1.963489e-02,-0.041879,-9.909925e-16,-0.027986,0.881597,-1.339746,-0.443302,...,-0.000057,-0.003424,-2.532024e-07,-0.000028,-0.000028,-0.000057,0.000000e+00,-0.000085,0.000000e+00,-0.126601


In [111]:
df2 = df2.set_axis(['Fluxes no constraints', 'Fluxes W', 'Fluxes E', 'Fluxes W-E'], axis='index')
df2

,NDPK5,SHK3Dr,NDPK6,NDPK8,OMPDC,PYNP2r,G5SD,CS,ICDHyr,PPA,...,PAI2I,FESD1s,I2FE2SR,TYRL,THZPSN3,PAI2T,APG3PAT120,4HTHRK,APH120,BIOMASS_Ec_iML1515_core_75p37M
Fluxes no constraints,0.000000e+00,0.334240,0.000000,5.929692e-16,0.290106,2.320660e-16,0.193865,6.913003,6.913003,3.070861,...,0.000393,0.003424,1.753994e-06,0.000196,0.000196,0.000393,1.731617e-17,0.000587,3.671980e-17,0.876997
Fluxes W,2.027348e-02,0.285991,0.019635,1.963489e-02,0.248228,0.000000e+00,0.165879,7.793504,5.572162,2.627562,...,0.000336,0.000000,1.500794e-06,0.000167,0.000167,0.000336,0.000000e+00,0.000502,0.000000e+00,0.750397
Fluxes E,-9.608357e-16,0.334241,0.000000,0.000000e+00,0.290107,9.909925e-16,0.193865,6.911908,6.911908,3.070865,...,0.000393,0.003424,1.753996e-06,0.000196,0.000196,0.000393,0.000000e+00,0.000587,0.000000e+00,0.876998
Fluxes W-E,2.027348e-02,-0.048250,0.019635,1.963489e-02,-0.041879,-9.909925e-16,-0.027986,0.881597,-1.339746,-0.443302,...,-0.000057,-0.003424,-2.532024e-07,-0.000028,-0.000028,-0.000057,0.000000e+00,-0.000085,0.000000e+00,-0.126601


In [113]:
sorted_row_E_W = df2.iloc[3].sort_values(ascending=False)
sorted_row_E_W

EX_h2_e       18.097180
PFL           18.095339
FHL           18.095028
FLDR2          5.624824
EX_co2_e       5.196399
                ...    
CYTBO3_4pp    -7.160069
NADH16pp      -8.136819
PDH           -9.688595
ATPS4rpp     -12.930086
EX_h2o_e     -16.243969
Name: Fluxes W-E, Length: 447, dtype: float64

In [125]:
model1.reactions.NADH16pp

Reaction identifier,NADH16pp
Name,NADH dehydrogenase (ubiquinone-8 & 3 protons) (periplasm)
Memory address,0x7f871e255d50
Stoichiometry,4.0 h_c + nadh_c + q8_c --> 3.0 h_p + nad_c + q8h2_c 4.0 H+ + Nicotinamide adenine dinucleotide - reduced + Ubiquinone-8 --> 3.0 H+ + Nicotinamide adenine dinucleotide + Ubiquinol-8
GPR,b2279 and b2277 and b2284 and b2282 and b2278 and b2288 and b2276 and b2287 and b2283 and b2281...
Lower bound,0.0
Upper bound,72.44110759315157


Getting the list of the most affected genes to search them on KeGG

In [138]:
reaction_names = sorted_row_E_W.index
gene_list =[]
for flux, reaction_names in zip(sorted_row_E_W,reaction_names):
    reaction_id = model1.reactions.get_by_id(reaction_names)
    genes = [gene.id for gene in reaction_id.genes]
    if flux >0:
        color= "blue"
    else:
        color= "#ffcccb"
    for gene in genes:
        gene_list.append(f"{gene} {color}")

In [139]:
reaction_id


Reaction identifier,EX_h2o_e
Name,H2O exchange
Memory address,0x7f871a7883d0
Stoichiometry,h2o_e <=> H2O H2O <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [140]:
gene_list

['b3952 blue',
 'b3951 blue',
 'b0902 blue',
 'b2579 blue',
 'b0903 blue',
 'b3114 blue',
 'b2487 blue',
 'b2724 blue',
 'b2482 blue',
 'b2489 blue',
 'b2721 blue',
 'b2490 blue',
 'b2484 blue',
 'b2719 blue',
 'b4079 blue',
 'b2488 blue',
 'b2481 blue',
 'b2486 blue',
 'b2723 blue',
 'b2722 blue',
 'b2485 blue',
 'b2483 blue',
 'b2720 blue',
 'b3924 blue',
 'b2895 blue',
 'b0684 blue',
 'b3236 blue',
 'b2029 blue',
 'b1852 blue',
 'b0767 blue',
 'b1854 blue',
 'b1676 blue',
 'b4015 blue',
 'b2976 blue',
 'b4014 blue',
 'b3386 blue',
 'b0723 blue',
 'b0721 blue',
 'b0722 blue',
 'b0724 blue',
 'b2465 blue',
 'b2935 blue',
 'b0008 blue',
 'b2464 blue',
 'b2465 blue',
 'b2935 blue',
 'b1611 blue',
 'b4122 blue',
 'b1612 blue',
 'b0720 blue',
 'b0118 blue',
 'b1276 blue',
 'b0118 blue',
 'b1276 blue',
 'b3781 blue',
 'b0888 blue',
 'b2582 blue',
 'b0474 blue',
 'b2518 blue',
 'b0474 blue',
 'b2518 blue',
 'b3781 blue',
 'b2234 blue',
 'b2582 blue',
 'b2235 blue',
 'b3781 blue',
 'b2234 bl

In [141]:
for t in gene_list:
    print(t)

b3952 blue
b3951 blue
b0902 blue
b2579 blue
b0903 blue
b3114 blue
b2487 blue
b2724 blue
b2482 blue
b2489 blue
b2721 blue
b2490 blue
b2484 blue
b2719 blue
b4079 blue
b2488 blue
b2481 blue
b2486 blue
b2723 blue
b2722 blue
b2485 blue
b2483 blue
b2720 blue
b3924 blue
b2895 blue
b0684 blue
b3236 blue
b2029 blue
b1852 blue
b0767 blue
b1854 blue
b1676 blue
b4015 blue
b2976 blue
b4014 blue
b3386 blue
b0723 blue
b0721 blue
b0722 blue
b0724 blue
b2465 blue
b2935 blue
b0008 blue
b2464 blue
b2465 blue
b2935 blue
b1611 blue
b4122 blue
b1612 blue
b0720 blue
b0118 blue
b1276 blue
b0118 blue
b1276 blue
b3781 blue
b0888 blue
b2582 blue
b0474 blue
b2518 blue
b0474 blue
b2518 blue
b3781 blue
b2234 blue
b2582 blue
b2235 blue
b3781 blue
b2234 blue
b2582 blue
b2235 blue
b0474 blue
b2518 blue
b3781 blue
b2234 blue
b2582 blue
b2235 blue
b0474 blue
b2518 blue
b3781 blue
b2234 blue
b2582 blue
b2235 blue
b2574 blue
b0123 blue
b3336 blue
b1818 blue
b1817 blue
b2415 blue
b2417 blue
b1819 blue
b1621 blue
b2416 blue